# quantum gates

In [1]:
include("utils.jl");

---
## Primitive gates
Quantum information process can be achived using a small set of operations (gates) combining the Pauli matrices $\mathbb{I}$, $\sigma_x$, $\sigma_y$, $\sigma_z$ (or simply: $\mathbb{I}$, $X$, $Y$, $Z$) and a two-qubits gates (usually the $C_{not}$ gate). Here I will call **primitive gate** any operation that cannot (or don't need) to be factorised and can be seem as a single instruction in a *quantum processor unit* (QPU).

In [2]:
# identity
id = Matrix(I,2,2)

xgate = [0 1;
         1 0]

ygate = [0 -im;
         im  0]

zgate = [1  0;
         0 -1];

### two-qubits gates
These gates can actually be implemented in terms of each other so, it's enough that one of them is available as a native operation on the QPU. Usually that means the $Z \otimes Z$ interaction since most of physical systems fall either in the Ising model
$$H_{ising} = - \sum_n \omega_n \sigma_n^z - \sum_{i,j} J_{ij} \sigma_i^z \sigma_j^z$$
or in the Heisenberg model
$$H_{heisenberg} = - \sum_n \omega_n \sigma_n^z - \sum_n J_n^x \sigma_n^x \sigma_{n+1}^x - \sum_n J_n^y \sigma_n^y \sigma_{n+1}^y - \sum_n J_n^z \sigma_n^z \sigma_{n+1}^z$$


In [3]:
# In both, Ising and Heisenberg model, the Z⊗Z interaction is
# consider part of the natural evolution of the given system.
zzgate = zgate ⊗ zgate

czgate = [1 0 0 0;
          0 1 0 0;
          0 0 1 0;
          0 0 0 -1]

cnotgate = [1 0 0 0;
            0 1 0 0;
            0 0 0 1;
            0 0 1 0]

# Although a swap operation is not a primitive gate, some platforms like
# trapped-ions allow physical swap which can be seem as a primitive operation.
swapgate = [1 0 0 0;
            0 0 1 0;
            0 1 0 0;
            0 0 0 1];

---
## Primitive rotations
In a QPU the standard qubit operations are in fact rotations $e^{-i\frac{\theta}{2} G}$ in the Bloch sphere where the factor ½ comes from the algebra of ½-spins. So, being more accurate, the real primitive operations for a single qubit are the rotations $R_X$, $R_Y$, $R_Z$ and for two-qubits it's enough have $R_{ZZ}$.

In [4]:
rx(θ) = exp(-im * θ/2 * xgate)
ry(θ) = exp(-im * θ/2 * ygate)
rz(θ) = exp(-im * θ/2 * zgate)

# As said before, the Z⊗Z interaction
# is a natural term in many platforms
rzz(θ) = exp(-im * θ/2 * zzgate);

---
## Factorising single-qubits gates
The standard gates $X$, $Y$, and $Z$ are implemented using rotations in the respective directions which also adds a global $\frac{\pi}{2}$-phases to the state,

$$R_X(\pi) = \begin{bmatrix}
0 & -i \\
-i & 0
\end{bmatrix} \implies X = e^{i\frac{\pi}{2}} R_X(\pi) ,$$

$$R_Y(\pi) = \begin{bmatrix}
0 & -1 \\
1 & 0
\end{bmatrix} \implies Y = e^{i\frac{\pi}{2}} R_Y(\pi) ,$$

$$R_Z(\pi) = \begin{bmatrix}
-i & 0 \\
0 & i
\end{bmatrix} \implies Z = e^{i\frac{\pi}{2}} R_Z(\pi) .$$

Since global phases are most of the time irrelevants we can simplify the definitions as
$$X \equiv R_X(\pi), \quad Z \equiv R_Z(\pi), \quad Z \equiv R_Z(\pi)$$

### Hadamard
Although it's one of the generator of the Clifford group, the standard Hadamard gate,
$$\mathrm{H} = \frac{1}{\sqrt{2}} \begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}, $$
is not a primitive gate. However, it can be seen as a $\frac{\pi}{2}$ rotation around $Y$,
$$R_Y(\pi/2) = \frac{1}{\sqrt{2}} \begin{bmatrix}
1 & -1 \\
1 & 1
\end{bmatrix} .$$
This rotation moves $|1\rangle \to -|-\rangle$. Usually this extra phase is irrelevante, but we can remove it by apply a $Z$ gate,
$$\mathrm{H} = \frac{1}{\sqrt{2}} \,\begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix} = e^{i\frac{\pi}{2}} R_Y (\pi/2) R_Z (\pi) .$$

Ommiting the global phase it can be reduces to
$$\mathrm{H} \equiv R_Y (\pi/2) R_Z (\pi) .$$

In [6]:
hadamard = global_phase(π/2) * ry(π/2) * rz(π)

hadamard |> real

2×2 Matrix{Float64}:
 0.707107   0.707107
 0.707107  -0.707107


### phase gate
Anothe generator of the Clifford group is the phase gate
$$S = \begin{bmatrix}
1 & 0 \\
0 & i
\end{bmatrix} = \sqrt{Z} = e^{i\frac{\pi}{4}} R_Z(\pi/2) \implies S \equiv R_Z(\pi/2) .$$

To construct a full universal gate set we also need the $T$ gate which is defined as $T = \sqrt{S}$ or $T \equiv R_Z(\pi/4)$.

In [43]:
sgate = global_phase(π/4) * rz(π/2)

sgate |> sparsefy

2×2 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+1.0im


### universal gates using rotations
A full set of universal gates can be created using $R_X$, $R_Y$, $R_Z$, and the _general phase gate_
$$P(\theta) = \begin{bmatrix}
1 & 0 \\
0 & e^{i\theta}
\end{bmatrix}$$
which can be written as $P(\theta) = e^{i\frac{\theta}{2}} R_Z(\theta)$. This implies that in order to have a full set of universal gates one needs only $R_X$, $R_Y$, $R_Z$, and $R_{ZZ}$.

In [45]:
pgate(θ) = global_phase(θ/2) * rx(θ);

---
## Factorising two-qubits gates

### $ZZ$ gate and two-qubits phase gate
Similar to the single-qubit gates, the $ZZ$ gate can be obtained by a $R_{ZZ}$ rotation meaning

$$ZZ = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & -1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix} = e^{i\frac{\pi}{2}} R_{ZZ}(\pi) \implies ZZ \equiv R_{ZZ}(\pi) .$$


In [32]:
zz_rzz = global_phase(π/2) * rzz(π)

zz_rzz |> real |> roundint 

4×4 Matrix{Int64}:
 1   0   0  0
 0  -1   0  0
 0   0  -1  0
 0   0   0  1


Analogous to the single-qubit phase gate $S$, we can define a two-quibits phase gate $\Pi$ as

$$\Pi = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & i & 0 & 0 \\
0 & 0 & i & 0 \\
0 & 0 & 0 & 1
\end{bmatrix} = \begin{bmatrix}
S & 0 \\
0 & iS^\dagger
\end{bmatrix} = \sqrt{ZZ} \implies \Pi \equiv R_{ZZ}(\pi/2) .$$

In [47]:
πgate = global_phase(π/4) * rzz(π/2)

πgate |> sparsefy

4×4 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+1.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+1.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  1.0+0.0im


### $C_{not}$ as a $C_Z$ interaction
This is a standard version since any controlled operation can be generated by a controlled-Z. In this case, the $C_{not}$ gate consists in a $C_Z$ enclosed by $R_Y(\pi/2)$ rotations on the target qubit,

$$C_{not} = \big[ \mathbb{I} \otimes R_Y(\pi/2) \big] \,
C_Z \,
\big[ \mathbb{I} \otimes R_Y(-\pi/2) \big] .$$

In [7]:
cnot_cz = (id ⊗ ry(π/2)) * czgate * (id ⊗ ry(-π/2))

cnot_cz |> real |> roundint 

4×4 Matrix{Int64}:
 1  0  0  0
 0  1  0  0
 0  0  0  1
 0  0  1  0

### $C_{not}$ as a $R_{ZZ}$ interaction
A more realistic implementation of the $C_{not}$ combines a $R_{ZZ}$ with a $R_X$ and $R_Y$ on the target qubit. We start considering the fact that
$$R_Y = SR_X S^\dagger\quad \text{and} \quad S^\dagger S^\dagger = Z .$$

We these two equalities we can write
$$R_Y(\theta/2) \,S\, R_X(-\theta/2) = S\,R_X(\theta/2)\,R_X(-\theta/2) = S$$
and
$$R_Y(\theta/2) \,iS^\dagger \, R_X(-\theta/2) = iS\,R_X(\theta/2)\,Z\,R_X(-\theta/2) = iS\,R_X(\theta)\,Z = iR_Y(\theta)\,ZS.$$

We can combine the both expression above in a matrix for,
$$\begin{bmatrix}
R_Y(\theta/2) & 0 \\
0 & R_Y(\theta/2) 
\end{bmatrix} \begin{bmatrix}
S & 0 \\
0 & iS^\dagger 
\end{bmatrix} \begin{bmatrix}
R_X(-\theta/2) & 0 \\
0 & R_X(-\theta/2) 
\end{bmatrix} = \begin{bmatrix}
S & 0 \\
0 & iR_Y(\theta)\,ZS 
\end{bmatrix} .$$

For $\theta=\pi$ we have $R_Y(\pi)\,Z=X$ which implies
$$\begin{bmatrix}
S & 0 \\
0 & iR_Y(\pi)\,ZS 
\end{bmatrix} = \begin{bmatrix}
\mathbb{I} & 0 \\
0 & X 
\end{bmatrix} \begin{bmatrix}
S & 0 \\
0 & iS 
\end{bmatrix} = \begin{bmatrix}
\mathbb{I} & 0 \\
0 & X 
\end{bmatrix} \begin{bmatrix}
\mathbb{I} & 0 \\
0 & i 
\end{bmatrix} \begin{bmatrix}
S & 0 \\
0 & S 
\end{bmatrix} .$$

Writing it in term of gates we find
$$\begin{bmatrix}
\mathbb{I} & 0 \\
0 & X 
\end{bmatrix} \begin{bmatrix}
\mathbb{I} & 0 \\
0 & i 
\end{bmatrix} \begin{bmatrix}
S & 0 \\
0 & S 
\end{bmatrix} = C_{not} \, (S \otimes S) .$$
As the final expression
$$\begin{aligned}
C_{not}
&= \big[\mathbb{I} \otimes R_Y(\pi/2) \big] \,\Pi\, \big[\mathbb{I} \otimes R_X(-\pi/2) \big] \, (S^\dagger \otimes S^\dagger) \\[.5em]
&= e^{-i\frac{\pi}{4}} \big[\mathbb{I} \otimes R_Y(\pi/2) \big] \,R_{ZZ}(\pi/2)\, \big[\mathbb{I} \otimes R_X(-\pi/2) \big] \, \big[ R_Z(-\pi/2) \otimes R_Z(-\pi/2) \big]
\end{aligned} .$$

In [123]:
cnot_zz = (id ⊗ ry(π/2)) * πgate * (id ⊗ rx(-π/2)) * (sgate' ⊗ sgate')

cnot_zz |> real |> roundint 

4×4 Matrix{Int64}:
 1  0  0  0
 0  1  0  0
 0  0  0  1
 0  0  1  0

### $C_Z$ as a $R_{ZZ}$ interaction
Starting from the $\Pi$ gate we have
$$\begin{bmatrix}
S & 0 \\
0 & iS^\dagger
\end{bmatrix} \implies \begin{bmatrix}
S & 0 \\
0 & iS^\dagger
\end{bmatrix} \begin{bmatrix}
S^\dagger & 0 \\
0 & S^\dagger
\end{bmatrix} = \begin{bmatrix}
\mathbb{I} & 0 \\
0 & iZ
\end{bmatrix} = C_Z \, (S \otimes \mathbb{I}) $$
thus,
$$C_Z
= \Pi \, \big(S^\dagger \otimes S^\dagger)
= e^{-i\frac{\pi}{4}} R_{ZZ}(\pi/2) \, \big[ R_Z(-\pi/2) \otimes R_Z(-\pi/2) \big] .$$

In [115]:
cz_zz = πgate * (sgate' ⊗ sgate')

cz_zz |> real |> roundint 

4×4 Matrix{Int64}:
 1  0  0   0
 0  1  0   0
 0  0  1   0
 0  0  0  -1

### $R_{XX}$, $R_{YY}$, $R_{XY}$, and $R_{YX}$ in terms of $R_{ZZ}$
Since we can easily move $Z$ to $X$ or $Y$ by appling
$$\mathrm{H}Z \mathrm{H} = X ,$$
$$S\mathrm{H}Z \mathrm{H}S^\dagger = Y ,$$
we can algebricaly perfom the simple transformations
$$\mathrm{H} e^{i \theta Z} \mathrm{H} = e^{i \theta X} ,$$
$$S\mathrm{H} e^{i \theta Z} \mathrm{H}S^\dagger = e^{i \theta Y} .$$

However, as said above in the section about single gates, the Hadamard (and the phase gate as well) can be used as simple rotantion around $Y$ (and $X$ respectively),
$$R_{XX}(\theta) = \big[ R_Y(\pi/2) \otimes R_Y(\pi/2) \big] \,
R_{ZZ}(\theta) \,
\big[ R_Y(-\pi/2) \otimes R_Y(-\pi/2) \big] ,$$
$$R_{YY}(\theta) = \big[ R_X(-\pi/2) \otimes R_X(-\pi/2) \big] \,
R_{ZZ}(\theta) \,
\big[ R_X(\pi/2) \otimes R_X(\pi/2) \big] .$$

In [13]:
rxx(θ) = (ry(π/2) ⊗ ry(π/2)) * rzz(θ) * (ry(-π/2) ⊗ ry(-π/2))

ryy(θ) = (rx(-π/2) ⊗ rx(-π/2)) * rzz(θ) * (rx(π/2) ⊗ rx(π/2));

Using the same principle we can create mixed rotarions $R_{XY}$ and $R_{YX}$,

$$R_{XY}(\theta) = \big[ R_Y(\pi/2) \otimes R_X(-\pi/2) \big] \,
R_{ZZ}(\theta) \,
\big[ R_Y(-\pi/2) \otimes R_X(\pi/2) \big] ,$$

$$R_{YX}(\theta) = \big[ R_X(-\pi/2) \otimes R_Y(\pi/2) \big] \,
R_{ZZ}(\theta) \,
\big[ R_X(\pi/2) \otimes R_Y(-\pi/2) \big] .$$

In [14]:
rxy(θ) = (ry(π/2) ⊗ rx(-π/2)) * rzz(θ) * (ry(-π/2) ⊗ rx(π/2))

ryx(θ) = (rx(-π/2) ⊗ ry(π/2)) * rzz(θ) * (rx(π/2) ⊗ ry(-π/2));

It is importante to notice that since
$$[XX,YY]=[XX,ZZ]=[YY,ZZ]=[XY,YX]=0$$
the three rotations $R_{XX}$, $R_{YY}$, $R_{ZZ}$ communte with each other as well as $R_{XY}$ with $R_{YX}$. 

### Swap gate as $R_{ZZ}$ interaction
Algebraically the swap gate can be written as
$$Swap =
\frac{1}{2} \left( \mathbb{I} + \vec{S}_1 \cdot \vec{S}_2 \right) =
\frac{1}{2} \left( \mathbb{I} + X_1 X_2 + Y_1 Y_2 + Z_1 Z_2  \right) .$$

When the $C_{not}$ can be found as a primitive operation, the swap gate can be written as three alternated $C_{not}$ gates similar to the swap operation using XOR ports in classical computers. An alternative construction can be found using $R_{ZZ}$ interactions.

Since the terms commute with each other we can start with a $R_{XX}$ rotation followed by a $R_{YY}$ rotation,
$$R_{XX}(\pi/2)\,R_{YY}(\pi/2) = \begin{bmatrix}
1 & 0 & 0 & -i \\
0 & 1 & -i & 0 \\
0 & -i & 1 & 0 \\
-i & 0 & 0 & 1
\end{bmatrix} \begin{bmatrix}
1 & 0 & 0 & i \\
0 & 1 & -i & 0 \\
0 & -i & 1 & 0 \\
i & 0 & 0 & 1
\end{bmatrix} = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & -i & 0 \\
0 & -i & 0 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix} .$$

and by applying a two-qubits phase gate we can obtain a $Swap$ gate
$$\begin{aligned}
Swap
&= R_{XX}(\pi/2)\,R_{YY}(\pi/2)\,\Pi \\[.5em]
&= e^{i\frac{\pi}{4}} \, R_{XX}(\pi/2) \, R_{YY}(\pi/2) \, R_{ZZ}(\pi/2)
\end{aligned} .$$

In [155]:
swap_zz = rxx(π/2) * ryy(π/2) * πgate

swap_zz |> real |> roundint 

4×4 Matrix{Int64}:
 1  0  0  0
 0  0  1  0
 0  1  0  0
 0  0  0  1

---
## Interactions with $\sigma^+$ and $\sigma^-$ using $R_{ZZ}$

### $\sigma^+ \sigma^- + \sigma^- \sigma^+$ (flip-flop)
This is a common rotation to simulate energy exchange because it excites one of the qubits in the same proportion it decays the other one. Since
$$\sigma^+ \sigma^- + \sigma^- \sigma^+ = \frac{XX + YY}{2}$$
we can write
$$e^{-i\theta (\sigma^+ \sigma^- + \sigma^- \sigma^+)} = R_{XX}(\theta)\,R_{YY}(\theta) .$$

In [18]:
flipflop_zz(θ) = rxx(θ) * ryy(θ)

#example
flipflop_zz(π/3) |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
  1.0-0.0im  -0.0-0.0im   -0.0-0.0im   0.0+0.0im
  0.0-0.0im   0.5-0.0im    0.0-0.87im  0.0+0.0im
  0.0-0.0im   0.0-0.87im   0.5-0.0im   0.0+0.0im
 -0.0-0.0im  -0.0+0.0im   -0.0+0.0im   1.0-0.0im

### $i\left(\sigma^+ \sigma^- - \sigma^- \sigma^+ \right)$ (asymmetric flip-flop)
Another useful rotation to simulation energy exchange is the asymmetric version of the flip-flop interaction
$$i \big(\sigma^+ \sigma^- - \sigma^- \sigma^+ \big) = \frac{XY - YX}{2}$$
which produces the rotation
$$e^{-i\theta \big[ i(\sigma^+ \sigma^- - \sigma^- \sigma^+) \big]} = R_{XY}(\theta)\,R_{YX}(-\theta) .$$

In [166]:
asym_flipflop_zz(θ) = rxy(θ) * ryx(-θ)

# example
asym_flipflop_zz(π/3) |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
  1.0+0.0im   -0.0+0.0im   0.0-0.0im   0.0+0.0im
  0.0+0.0im    0.5+0.0im  0.87+0.0im  -0.0+0.0im
 -0.0-0.0im  -0.87+0.0im   0.5-0.0im   0.0-0.0im
  0.0-0.0im    0.0+0.0im  -0.0-0.0im   1.0-0.0im

### $\sigma^+ \sigma^+ + \sigma^- \sigma^-$ (sync)
Analogous to the flip-flop interaction is the $\sigma^+ \sigma^+ + \sigma^- \sigma^-$ which rotations two qubits when they are in the same state
$$\sigma^+ \sigma^+ + \sigma^- \sigma^- = \frac{XX - YY}{2} .$$

Such interaction produces the following rotation
$$e^{-i\theta (\sigma^+ \sigma^+ + \sigma^- \sigma^+)} = R_{XX}(\theta)\,R_{YY}(-\theta) .$$

In [19]:
sync_zz(θ) = rxx(θ) * ryy(-θ)

sync_zz(π/3) |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
  0.5-0.0im   0.0-0.0im  0.0-0.0im   0.0-0.87im
 -0.0-0.0im   1.0+0.0im  0.0+0.0im  -0.0+0.0im
 -0.0-0.0im   0.0+0.0im  1.0+0.0im  -0.0+0.0im
 -0.0-0.87im  0.0+0.0im  0.0+0.0im   0.5+0.0im

### $i\left(\sigma^+ \sigma^+ - \sigma^- \sigma^- \right)$ (asymmetric sync)
As the flip-flop operations, the synchronous rotations also has an asymmetric version
$$i \big( \sigma^+ \sigma^+ - \sigma^- \sigma^- \big) = \frac{XY + YX}{2}$$
which results in
$$e^{-i\theta \big[ i(\sigma^+ \sigma^+ - \sigma^- \sigma^-) \big]} = R_{XY}(\theta)\,R_{YX}(\theta) .$$

In [21]:
asym_sync_zz(θ) = rxy(θ) * ryx(θ)

asym_sync_zz(π/3) |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
  0.5+0.0im  -0.0-0.0im  -0.0-0.0im  -0.87+0.0im
  0.0-0.0im   1.0+0.0im  -0.0-0.0im    0.0+0.0im
  0.0-0.0im  -0.0+0.0im   1.0+0.0im    0.0+0.0im
 0.87-0.0im  -0.0+0.0im  -0.0+0.0im    0.5+0.0im

## References
_**TODO...**_